In [9]:
# GPU kontrolü
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
Memory: 14.74 GB


In [10]:
# Colab'a dosyaları yükle
import os

print("Current directory:", os.getcwd())

# Eğer /content değilse, muhtemelen lokal çalışıyorsun
if os.getcwd() == '/content':
    print("\n✓ Colab ortamındasın")
    print("\nŞimdi sol taraftaki Files ikonundan tüm proje klasörünü yükle:")
    print("  1. Sol menüden 'Files' seç")
    print("  2. Upload butonuna tıkla") 
    print("  3. src/, main.py, requirements.txt vb. tüm dosyaları yükle")
    print("\nYa da GitHub'dan clone yap:")
    print("  !git clone <your-repo-url>")
else:
    print(f"\n✓ Lokal ortamdasın: {os.getcwd()}")

print("\nDosyalar:")
!ls -la

Current directory: /content

✓ Colab ortamındasın

Şimdi sol taraftaki Files ikonundan tüm proje klasörünü yükle:
  1. Sol menüden 'Files' seç
  2. Upload butonuna tıkla
  3. src/, main.py, requirements.txt vb. tüm dosyaları yükle

Ya da GitHub'dan clone yap:
  !git clone <your-repo-url>

Dosyalar:
total 20
drwxr-xr-x 1 root root 4096 Nov 29 20:24 .
drwxr-xr-x 1 root root 4096 Nov 29 19:58 ..
drwxr-xr-x 4 root root 4096 Nov 20 14:30 .config
drwxr-xr-x 3 root root 4096 Nov 29 20:24 outputs
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data


## Kurulum (Sadece Colab için - bir kez çalıştır)

In [ ]:
# Colab'da gerekli kütüphaneleri yükle
!pip install -q transformers datasets torch scikit-learn tqdm

print("✓ Kütüphaneler yüklendi!")

Running: /usr/bin/python3 main.py --model bert-base-uncased --epochs 1 --batch_size 8 --learning_rate 2e-5 --device cpu
STDOUT:


STDERR:
/usr/bin/python3: can't open file '/content/main.py': [Errno 2] No such file or directory


Exit code: 2


In [ ]:
# GitHub'dan projeyi klonla
!git clone https://github.com/KULLANICI_ADIN/cti-bert-ttp.git
%cd cti-bert-ttp

# Dosyaları kontrol et
!ls -la

print("\n✓ Proje dosyaları yüklendi!")

## Option 1: main.py ile çalıştır (Önerilen)

In [ ]:
# BERT-base ile eğitim (3 epoch, batch 16)
!python main.py \
    --model bert-base-uncased \
    --epochs 3 \
    --batch_size 16 \
    --learning_rate 2e-5 \
    --device cuda

ModuleNotFoundError: No module named 'src'

In [ ]:
# SecBERT (security domain model) ile eğitim
!python main.py \
    --model jackaduma/SecBERT \
    --epochs 3 \
    --batch_size 16 \
    --device cuda

In [ ]:
## Option 2: Modüler kullanım (interaktif)

In [ ]:
# Import modülleri
import sys
import os
import torch

# Project root'u Python path'e ekle
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Working directory: {os.getcwd()}")
print(f"GPU available: {torch.cuda.is_available()}")

from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
import json
from datetime import datetime

print("✓ All modules imported successfully!")

In [ ]:
# Konfigürasyon
MODEL_NAME = "bert-base-uncased"  # veya "jackaduma/SecBERT"
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
MAX_LENGTH = 512
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"./outputs/{MODEL_NAME.replace('/', '_')}_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Device: {DEVICE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Output: {output_dir}")

In [ ]:
# Data hazırlama
data = prepare_data(
    model_name=MODEL_NAME,
    max_length=MAX_LENGTH
)

train_dataset = data['train_dataset']
test_dataset = data['test_dataset']
label_list = data['label_list']
num_labels = data['num_labels']

# Save labels
with open(os.path.join(output_dir, 'labels.json'), 'w') as f:
    json.dump(label_list, f, indent=2)
    
print(f"✓ Data loaded: {len(train_dataset)} train, {len(test_dataset)} test")

## Görselleştirme

In [ ]:
import matplotlib.pyplot as plt

# Training loss
plt.figure(figsize=(10, 6))
plt.plot(history['epoch'], history['train_loss'], marker='o', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(output_dir, 'training_loss.png'), dpi=300, bbox_inches='tight')
plt.show()

# Metrics
metrics_names = ['Micro F1', 'Macro F1', 'Samples F1']
metrics_values = [
    metrics['micro_f1'],
    metrics['macro_f1'],
    metrics['samples_f1']
]

plt.figure(figsize=(10, 6))
plt.bar(metrics_names, metrics_values, color=['#3498db', '#2ecc71', '#f39c12'])
plt.ylabel('Score')
plt.title('Performance Metrics')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)

for i, v in enumerate(metrics_values):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

plt.savefig(os.path.join(output_dir, 'metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

## Sonuçları İndir (Colab'da)

In [ ]:
# Tüm output klasörünü zip'le ve indir
import subprocess
import shutil

# Create zip file
shutil.make_archive('results', 'zip', output_dir)
print(f"Created results.zip from {output_dir}")

# Colab'da indir
try:
    from google.colab import files
    files.download('results.zip')
except ImportError:
    print("Not in Colab, zip file saved locally as results.zip")